# TEXT PREPROCESSING PIPELINE (MOST IMPORTANT)

In [29]:
import nltk                # The main NLP library
import re                  # Regular Expressions - for finding patterns in text
from nltk.corpus import stopwords    # Common words like "the", "is", "and"
from nltk.stem import WordNetLemmatizer  # Converts words to base form

# Downloads needed data
nltk.download('punkt')      # Tokenizer (splits text into words)
nltk.download('stopwords')  # List of common words to remove
nltk.download('wordnet')    # Dictionary for lemmatization

[nltk_data] Downloading package punkt to
[nltk_data]     /home/e2727bef-27c9-47fa-850c-
[nltk_data]     5006ef847018/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/e2727bef-27c9-47fa-850c-
[nltk_data]     5006ef847018/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/e2727bef-27c9-47fa-850c-
[nltk_data]     5006ef847018/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [30]:
def preprocess_text(text):
    """
    Real-world text cleaning pipeline
    """
    # 1. Lowercase
    text = text.lower()

    # 2. Remove URLs, mentions, hashtags (for social media)
    text = re.sub(r'http\S+|@\w+|#\w+', '', text)

    # 3. Remove numbers and special chars (keep words only)
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # 4. Tokenize
    tokens = nltk.word_tokenize(text)

    # 5. Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # 6. Lemmatization (better than stemming for DS)
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # 7. Remove short words (<2 chars)
    tokens = [word for word in tokens if len(word) > 2]

    return tokens

# Test it
sample = "Check out this tutorial at https://example.com! #NLP is awesome 100%"
print("Cleaned tokens:", preprocess_text(sample))

Cleaned tokens: ['check', 'tutorial', 'awesome']


# FEATURE EXTRACTION (For ML Models)

In [31]:
from nltk import FreqDist
from collections import Counter
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/e2727bef-27c9-47fa-850c-
[nltk_data]     5006ef847018/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to
[nltk_data]     /home/e2727bef-27c9-47fa-850c-
[nltk_data]     5006ef847018/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


True

In [32]:

def extract_features(text_list):
    """
    Convert text to ML-ready features
    """
    features = []
    
    for text in text_list:
        tokens = preprocess_text(text)
        
        # Feature 1: Basic counts
        features_dict = {
            'word_count': len(tokens),
            'char_count': len(text),
            'avg_word_length': sum(len(w) for w in tokens)/len(tokens) if tokens else 0   #Average letters per word -> ["cat", "elephant"] → (3+8)/2 = 5.5
        }
        
        # Feature 2: Word frequencies (top 10)
        freq_dist = FreqDist(tokens)
        top_words = freq_dist.most_common(5)
        
        # Feature 3: POS ratios
        tagged = nltk.pos_tag(tokens)
        pos_counts = Counter(tag for word, tag in tagged)
        
        if len(tokens) > 0:
            features_dict.update({
                'noun_ratio': pos_counts.get('NN', 0)/len(tokens),
                'verb_ratio': sum(pos_counts.get(tag, 0) for tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'])/len(tokens),
                'unique_word_ratio': len(set(tokens))/len(tokens)
            })
        
        features.append(features_dict)
    
    return features

# Example
reviews = [
    "Amazing product! Highly recommended.",
    "Terrible experience, would not buy again.",
    "It's okay, nothing special."
]

features = extract_features(reviews)
for i, feat in enumerate(features):
    print(f"Review {i+1} features: {feat}")

Review 1 features: {'word_count': 4, 'char_count': 36, 'avg_word_length': 7.75, 'noun_ratio': 0.25, 'verb_ratio': 0.5, 'unique_word_ratio': 1.0}
Review 2 features: {'word_count': 4, 'char_count': 41, 'avg_word_length': 6.5, 'noun_ratio': 0.25, 'verb_ratio': 0.25, 'unique_word_ratio': 1.0}
Review 3 features: {'word_count': 3, 'char_count': 27, 'avg_word_length': 6.0, 'noun_ratio': 0.6666666666666666, 'verb_ratio': 0.0, 'unique_word_ratio': 1.0}


# SENTIMENT ANALYSIS (Most Common DS Task)

In [33]:
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')  # VADER stands for: Valence Aware Dictionary and sEntiment Reasoner

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/e2727bef-27c9-47fa-850c-
[nltk_data]     5006ef847018/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [34]:
def analyze_sentiments(texts):
    """
    Analyze positive/negative sentiment
    """
    sia = SentimentIntensityAnalyzer()
    results = []
    
    for text in texts:
        scores = sia.polarity_scores(text)
        # Compound score: -1 (very negative) to +1 (very positive)
        sentiment = "positive" if scores['compound'] > 0.05 else "negative" if scores['compound'] < -0.05 else "neutral"
        
        results.append({
            'text': text[:50] + "...",  # Preview
            'sentiment': sentiment,
            'compound_score': scores['compound'],
            'pos_score': scores['pos'],
            'neg_score': scores['neg']
        })
    
    return results

# Real-world example
customer_reviews = [
    "The product works great and delivery was fast!",
    "Poor quality, broke after 2 days. Very disappointed.",
    "It's okay for the price, but could be better.",
    "Absolutely love it! Best purchase ever!!!"
]

sentiments = analyze_sentiments(customer_reviews)
for result in sentiments:
    print(f"{result['sentiment'].upper():10} | Score: {result['compound_score']:.3f} | {result['text']}")

POSITIVE   | Score: 0.659 | The product works great and delivery was fast!...
NEGATIVE   | Score: -0.852 | Poor quality, broke after 2 days. Very disappointe...
POSITIVE   | Score: 0.649 | It's okay for the price, but could be better....
POSITIVE   | Score: 0.903 | Absolutely love it! Best purchase ever!!!...


# TOPIC MODELING (LDA - Super Useful)

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

def extract_topics(documents, n_topics=3, n_words=5):
    """
    Discover hidden topics in documents - FIXED VERSION
    """
    # 1. Preprocess all documents
    processed_docs = [' '.join(preprocess_text(doc)) for doc in documents]
    
    print(f"📊 We have {len(documents)} documents")
    print(f"📝 Sample processed: {processed_docs[0][:50]}...")
    
    # 2. Create document-term matrix - WITH FIXED PARAMETERS!
    vectorizer = CountVectorizer(
        max_df=0.95,      # Remove words in >95% of docs (too common)
        min_df=1,         # CHANGED FROM 2 TO 1! (keep words that appear at least once)
        max_features=100  # CHANGED FROM 1000! (for small dataset)
    )
    
    dtm = vectorizer.fit_transform(processed_docs)
    
    print(f"✅ Created matrix: {dtm.shape[0]} docs × {dtm.shape[1]} words")
    
    # 3. Check if we have enough words
    if dtm.shape[1] < n_topics:
        print(f"⚠️  Warning: Only {dtm.shape[1]} words for {n_topics} topics")
    
    # 4. Apply LDA
    lda = LatentDirichletAllocation(
        n_components=min(n_topics, dtm.shape[1]),  # Safety: topics ≤ words
        random_state=42,
        learning_method='online'
    )
    
    lda.fit(dtm)
    
    # 5. Display topics
    feature_names = vectorizer.get_feature_names_out()
    
    topics = []
    for topic_idx, topic in enumerate(lda.components_):
        top_words_idx = topic.argsort()[:-n_words-1:-1]
        top_words = [feature_names[i] for i in top_words_idx]
        topics.append(f"Topic {topic_idx+1}: {' | '.join(top_words)}")
    
    return topics

# Example with news headlines
headlines = [
    "Stock market reaches all time high as tech companies soar",
    "New AI model achieves breakthrough in language understanding",
    "Federal Reserve considers interest rate hike to curb inflation",
    "Deep learning algorithms revolutionize medical diagnosis",
    "Economists predict recession amid global uncertainties",
    "Neural networks show remarkable progress in computer vision"
]

topics = extract_topics(headlines, n_topics=2)
for topic in topics:
    print(topic)

📊 We have 6 documents
📝 Sample processed: stock market reach time high tech company soar...
✅ Created matrix: 6 docs × 41 words
Topic 1: company | global | soar | recession | stock
Topic 2: vision | show | reserve | hike | federal


# TEXT CLASSIFICATION (Complete Pipeline)

In [36]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [37]:
def train_text_classifier(texts, labels, test_size=0.2):
    """
    End-to-end text classifier
    """
    # 1. Split data
    X_train, X_test, y_train, y_test = train_test_split(
        texts, labels, test_size=test_size, random_state=42
    )
    
    # 2. Feature extraction (TF-IDF)
    vectorizer = TfidfVectorizer(
        max_features=1000,
        stop_words='english',
        ngram_range=(1, 2)  # Include bigrams
    )
    
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)
    
    # 3. Train classifier
    classifier = MultinomialNB()
    classifier.fit(X_train_vec, y_train)
    
    # 4. Evaluate
    y_pred = classifier.predict(X_test_vec)
    
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    
    # 5. Return everything for reuse
    return {
        'vectorizer': vectorizer,
        'classifier': classifier,
        'accuracy': classifier.score(X_test_vec, y_test)
    }

# Example: Spam vs Ham classification
messages = [
    "Win a free iPhone! Click here now!",
    "Meeting scheduled for tomorrow at 3 PM",
    "Congratulations! You've won $1000 prize!",
    "Don't forget to submit your report by Friday",
    "Limited time offer! Buy one get one free!",
    "Can we reschedule our call to next week?"
]

labels = ['spam', 'ham', 'spam', 'ham', 'spam', 'ham']  # Corresponding labels

model = train_text_classifier(messages, labels)

# Test with new message
new_message = ["Urgent! Claim your prize now!"]
new_vec = model['vectorizer'].transform(new_message)
prediction = model['classifier'].predict(new_vec)
print(f"\nNew message prediction: {prediction[0]}")

Classification Report:
              precision    recall  f1-score   support

         ham       1.00      1.00      1.00         1
        spam       1.00      1.00      1.00         1

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2


New message prediction: spam


# NAMED ENTITY RECOGNITION (For Information Extraction)

In [38]:
from nltk import ne_chunk
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/e2727bef-27c9-47fa-850c-
[nltk_data]     5006ef847018/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /home/e2727bef-27c9-47fa-850c-
[nltk_data]     5006ef847018/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [39]:
def extract_entities(text):
    """
    Extract people, organizations, locations
    """
    # Tokenize and POS tag
    tokens = nltk.word_tokenize(text)
    tagged = nltk.pos_tag(tokens)
    
    # Apply NER
    chunks = ne_chunk(tagged)
    
    entities = {
        'PERSON': [],
        'ORGANIZATION': [],
        'LOCATION': []
    }
    
    # Extract entities
    for chunk in chunks:
        if hasattr(chunk, 'label'):
            entity_type = chunk.label()
            entity_name = ' '.join(c[0] for c in chunk)
            
            if entity_type in entities:
                entities[entity_type].append(entity_name)
    
    return entities

# Example from news article
news_text = """
Apple CEO Tim Cook announced new products at the event in California.
Microsoft and Google are also investing heavily in AI research.
Elon Musk's SpaceX launched satellites from Florida last week.
"""

entities = extract_entities(news_text)
print("📌 Named Entities Found:")
for entity_type, names in entities.items():
    if names:
        print(f"{entity_type}: {', '.join(set(names))}")

📌 Named Entities Found:
PERSON: Elon Musk, Microsoft, Apple
ORGANIZATION: CEO Tim Cook, SpaceX
